<a href="https://colab.research.google.com/github/gfeyzakorkmaz/gfeyzakorkmaz/blob/main/pythonday51.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
path = '/content/drive/MyDrive/Dataset/complex_marketing_data_enriched.xlsx'

In [3]:
df = pd.read_excel(path)

In [10]:
from sklearn.cluster import KMeans
import pandas as pd


X = df[['Age', 'Total Spent', 'Customer Tenure (Years)', 'Discount Availed']]

kmeans = KMeans(n_clusters=3, random_state=42)
df['Customer Segment'] = kmeans.fit_predict(X)

df[['Customer ID', 'Customer Segment']]

,Customer ID,Customer Segment
0,1,0
1,2,1
2,3,1
3,4,0
4,5,2
5,6,2
6,7,1
7,8,0
8,9,1
9,10,1


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = df[['Age', 'Total Spent', 'Discount Availed', 'Website Visit Frequency']]
y = df['Converted']

X_train, X_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

df['Conversion Probability'] = model.predict_proba(X)[:, 1]

df[['Customer ID', 'Conversion Probability']]

,Customer ID,Conversion Probability
0,1,0.485172
1,2,0.971135
2,3,0.445070
3,4,0.004311
4,5,0.684639
5,6,0.431964
6,7,0.614066
7,8,0.858141
8,9,0.207868
9,10,0.975747


In [11]:
regional_sales = df.groupby('Region').agg({
    'Total Spent': 'sum',
    'Discount Availed': 'mean'
    }).reset_index()

regional_sales

,Region,Total Spent,Discount Availed
0,East,3170,36.666667
1,North,1940,29.750000
2,South,750,72.000000
3,West,3060,45.166667


In [14]:
from textblob import TextBlob

df['Sentiment Score'] = df['Comments'].apply(lambda x: TextBlob(x).sentiment.polarity)

df[['Customer ID', 'Comments', 'Sentiment Score']]


,Customer ID,Comments,Sentiment Score
0,1,Satisfied with the service,0.500000
1,2,Needs improvement in delivery,0.000000
2,3,Regular customer,0.000000
3,4,Occasional buyer,0.000000
4,5,High spender,0.160000
5,6,Average experience,-0.150000
6,7,Prefers online shopping,0.000000
7,8,New customer,0.136364
8,9,Loyal to the brand,0.333333
9,10,High conversion rate,0.160000


In [15]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

stock_series = df.groupby('Purchased Category')['Stock'].sum()

model = ExponentialSmoothing(stock_series, trend='add', seasonal=None)
fit = model.fit()

stock_predictions = pd.DataFrame({
    'Category': stock_series.index,
    'Predicted Stock': fit.fittedvalues
})

stock_predictions

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:559: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  _index = to_datetime(index)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)


,Category,Predicted Stock
Purchased Category,,
Beauty,Beauty,144.200014
Books,Books,170.500009
Clothing,Clothing,196.800009
Electronics,Electronics,223.100003
Home Goods,Home Goods,249.400000


In [16]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X, y)

feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

feature_importance

,Feature,Importance
0,Age,0.495012
1,Total Spent,0.227145
2,Discount Availed,0.160710
3,Website Visit Frequency,0.117133


In [17]:
df['ROI'] = (df['Total Spent'] - df['Discount Availed']) / df['Total Spent']

campaign_roi = df.groupby('Campaign')['ROI'].mean().reset_index()

campaign_roi


,Campaign,ROI
0,Black Friday,0.897333
1,Summer Sale,0.845167
2,Winter Sale,0.882974


In [18]:
gender_behavior = df.groupby('Gender').agg({
    'Total Spent': ['mean', 'sum'],
    'Discount Availed': 'mean'
}).reset_index()

gender_behavior

Gender Total Spent       Discount Availed
                mean   sum             mean
0      F  467.272727  5140        35.454545
1      M  420.000000  3780        44.666667

In [20]:
correlation_matrix = df[['Total Spent', 'Age', 'Discount Availed', 'Customer Tenure (Years)' ]].corr()

correlation_matrix

,Total Spent,Age,Discount Availed,Customer Tenure (Years)
Total Spent,1.000000,0.244955,0.283919,0.097870
Age,0.244955,1.000000,0.004612,0.409548
Discount Availed,0.283919,0.004612,1.000000,-0.154477
Customer Tenure (Years),0.097870,0.409548,-0.154477,1.000000


In [25]:
Q1 = df['Total Spent'].quantile(0.25)
Q3 = df['Total Spent'].quantile(0.75)
IQR = Q3 - Q1

outliers = df[(df['Total Spent'] < Q1 - 0.2 * IQR) | (df['Total Spent'] > Q3 + 0.2 * IQR)]
outliers

,Customer ID,Age,Gender,Total Spent,Converted,Comments,Purchased Category,Stock,Region,Income Level,Discount Availed,Customer Tenure (Years),Campaign,Return Rate (%),Website Visit Frequency,Average Cart Value,Customer Segment,Conversion Probability,Sentiment Score,ROI
0,1,23,F,150,1,Satisfied with the service,Clothing,4,East,Medium,46,4,Winter Sale,2.981829,3,378.00,0,0.485172,0.500000,0.693333
2,3,45,F,800,0,Regular customer,Books,24,North,Medium,50,12,Black Friday,6.547401,0,250.69,1,0.445070,0.000000,0.937500
3,4,52,M,200,0,Occasional buyer,Beauty,10,East,Medium,54,7,Winter Sale,3.923717,1,222.58,0,0.004311,0.000000,0.730000
6,7,38,F,670,1,Prefers online shopping,Home Goods,69,North,Low,50,8,Winter Sale,2.738895,1,290.23,1,0.614066,0.000000,0.925373
7,8,27,M,120,1,New customer,Home Goods,0,North,Low,6,3,Winter Sale,5.268858,7,181.40,0,0.858141,0.136364,0.950000
9,10,33,M,750,1,High conversion rate,Clothing,45,South,Medium,72,1,NaN,6.775497,4,237.93,1,0.975747,0.160000,0.904000
17,18,35,M,700,1,Satisfied with customer service,Clothing,87,West,High,89,6,NaN,4.179233,5,440.44,1,0.972294,0.500000,0.872857
